# Twitter Scraper 

python modules
* différents modules python  tweepy, twint, twitterscraper https://larevueia.fr/nlp-avec-python-analyse-de-sentiments-sur-twitter/
* credentials pour utiliser l'API de twitter https://developer.twitter.com/en/portal/apps/8475429/keys
* module python pour interfacer l'API tweepy api https://docs.tweepy.org/en/latest/api.html ; https://docs.tweepy.org/en/latest/api.html#API.search ; https://docs.tweepy.org/en/latest/cursor_tutorial.html?highlight=cursor#cursor-tutorial
* https://github.com/JustAnotherArchivist/snscrape

Twitter et TAL
* prétraiter des textes issus de twitter https://github.com/cbaziotis/ekphrasis


## twitterscraper 

api non fonctionnelle

In [4]:
from twitterscraper import query_tweets
import datetime as dt
import pandas as pd

debut = dt.date(2021,1,1)
fin = dt.date(2021,12,31)
#mots="Covid-19 OR Covid OR Corona OR Pandémie OR épidémie OR Coronavirus OR virus"
mots = "😠"
mots = "colère"

tweets = query_tweets(query=mots, begindate = debut, enddate = fin, lang = "fr")

tweets = pd.DataFrame(t.__dict__ for t in tweets)

tweets.to_csv('built/tweet_angry.csv')

ModuleNotFoundError: No module named 'twitterscraper'

##  tweepy

`sudo pip3 install tweepy`

* Require a developer twitter account to obtain API/Consumer key and secret as well as access token key and secret. 
* Require a premium access to go further than the 7-day limit of the search index.

See https://developer.twitter.com/en/portal/apps/8475429/keys

In [5]:
import os
import tweepy as tw

consumer_key = "SET_ME" 
consumer_secret = "SET_ME"
access_token_key = "SET_ME"
access_token_secret = "SET_ME"

# Authentification :

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

requete = "😠"# "Covid-19 OR Covid OR Corona OR Pandémie OR épidémie OR Coronavirus OR virus"

tweets = tw.Cursor(api.search,
                   q = requete,
                   lang = "fr",
                   since='2018-01-01').items(1000)

all_tweets = [tweet.text for tweet in tweets]

ModuleNotFoundError: No module named 'tweepy'

In [ ]:
all_tweets

In [ ]:
entities = [tweet.entities for tweet in tweets]
print (entities)

## twint

* main page https://github.com/twintproject/twint
* documentation: configuration options, tweets attributes... https://github.com/twintproject/twint/wiki/Configuration
* tutoriel avec api https://towardsdatascience.com/analyzing-tweets-with-nlp-in-minutes-with-spark-optimus-and-twint-a0c96084995f

  `sudo pip install twint`


### How to use twint ?

#### .Limit, .Since .Until options

Twint is said to be slow but this is not the main issue.

From the [Twint API](https://github.com/twintproject/twint/wiki/Configuration), we can read that Twint allows to
* set a limit of number of Tweets to pull. 
* filter Tweets sent since date, works only with twint.run.Search (Example: 2017-12-27).
* filter Tweets sent until date, works only with twint.run.Search (Example: 2017-12-27).

When successive Search requests are run, if the limit of tweets to pull is too low or the period is too short, you may face the `CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'` issue. The issue stops the collect process. It seems to occur when the time between successive Search requests is repeatedly too short.

Setting a limit too high on a short period may be not the solution when there are not so many tweets to fetch (i.e. `[!] No more data! Scraping will stop now.`). 

I recommend to test and determine how many tweets can be fetched on a certain time slice and consequently to conclude what is the best time slice for your resquest to fetch. 

For example, when setting a limit of 1000 with a period of 1 day (and repeating the search on several successive days), you obtain 100 hundreds tweets for the whole period, then you may set a period of 10 * 1 days (or a bit more).

#### Search request options

various kind of resquests:
``` 
request = "😠 OR 😡 OR 😤"
request = "😠 AND 😡 AND 😤" 
request = "Banksy" 
request = "Un Banksy vendu plus de 19 millions d’euros, une somme qui sera reversée au service de santé britannique" 
``` 

#### How to focus on a specific language ? 

* `tw.lang='fr'`
doesnt not prevent to retrieve tweets in other language. The tweet lang attribute helps to filter after fetching.

* `tw.Geo = "46.6055983, 1.8750922, 545km" `                  (string) - Geo coordinates (lat,lon,km/mi.) ; 
https://fr.wikipedia.org/wiki/Centre_de_la_France # 543,7 km ;  Centre du cercle minimum 
Not convinced by the effect of the distance specification.

* `tw.Near = "Paris"`
Seems work pretty well. But introduce a bias. "Paris" is one of the tweet's topics.


### Issues

#### `CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)`

https://github.com/twintproject/twint/issues/1063

```
sudo pip3 uninstall twint
sudo pip3 install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
```

#### `RuntimeError: This event loop is already running`

Solve compatibility issues with notebooks and RunTime errors.

```
sudo pip3 install nest_asyncio
```

Then in python code:
```
import nest_asyncio
nest_asyncio.apply()
```

#### `CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'`

When too many requests with no pause between them. A pause can be created simply by increasing the limit of the tweets to fetch. 

Approximatively we note that a limit set to 1000 last 10 seconds.

### Fetching some tweets

In [ ]:
# package
import twint
import nest_asyncio
import time
import pandas as pd
import datetime
import calendar
import os

def add_time_delta(a_date):
    #a_date += datetime.timedelta(minutes=15)
    #a_date += datetime.timedelta(days=1)
    a_date += datetime.timedelta(weeks=1)
    return a_date
#
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.datetime(year, month, day,sourcedate.hour, sourcedate.minute, sourcedate.second)

# emotions
anger = "😣 OR 😩 OR 😫 OR 💔 OR 😞 OR ‼ OR 👊 OR 🔫 OR 💥 OR ❗ OR 🔥 OR 🙅 OR 👎 OR ❌ OR 😒 OR 😤 OR 😡 OR 😠"
anticipation = "🎀 OR 😉 OR ✌ OR 😻 OR 😊 OR ☺ OR 😜 OR 😕 OR 😱 OR ‼ OR ❗ OR 🍻 OR 😚 OR ✈ OR 🙏 OR 🙊 OR 😅 OR 😈 OR 😓 OR 😟 OR 😋 OR 😙 OR 💓 OR ✊ OR 🙋 OR 😬 OR 💰 OR 💭 OR 👀"
disgust = "💔 OR 😶 OR 😨 OR 😳 OR 😭 OR 😢 OR 😕 OR 😰 OR 😱 OR 😪 OR 😟 OR 😑 OR 😞 OR 😩 OR 😷 OR 🙅 OR 😡 OR 😠 OR ❌ OR 😤 OR 😫 OR 😒 OR 💩 OR 😣 OR 😖 OR 👎"
fear = "😞 OR 😭 OR 😥 OR 🙅 OR 😶 OR 😕 OR 👀 OR ❗ OR 😬 OR 😣 OR 😩 OR 🙊 OR 😷 OR 😢 OR 😖 OR 😳 OR 💀 OR 😓 OR 👻 OR 😟 OR 😰 OR 😱 OR 😨"
joy = "💘 OR 💃 OR 😛 OR 😀 OR 🌈 OR 💐 OR ❤ OR 🙌 OR 😇 OR 🍻 OR ♥ OR 😘 OR 😎 OR 😻 OR 😋 OR 🌞 OR 💋 OR 😌 OR 😝 OR 😃 OR 😍 OR 💞 OR 💕 OR 😙 OR 😄 OR 😚 OR 😁 OR 💗 OR 💖 OR 🎉 OR 😊 OR 😂 OR 😹 OR 😆 OR ☺"
sadeness = "😓 OR 😡 OR 😕 OR 😒 OR 😖 OR 😨 OR 😪 OR 😣 OR 😰 OR 😟 OR 😔 OR 😫 OR 😩 OR 😥 OR 😞 OR 💔 OR 😭 OR 😢" # 15 minutes sur Paris
surprise = "⚡ OR 💐 OR 🎀 OR 😅 OR 🔥 OR 🙌 OR 😨 OR ☺ OR 🙊 OR ✨ OR 😳 OR 😍 OR 🎉 OR 🙈 OR 👀 OR 😱 OR ❗ OR ‼"
trust = "😊 OR 🙌 OR ☑ OR ✌ OR 👍 OR 😉 OR 💎 OR 💗 OR 😇 OR 😍 OR ☺ OR 🍻 OR 💖 OR 👏 OR 💯 OR 👌 OR ♥ OR 💐 OR 🙏 OR 💞 OR 😌 OR 😘 OR 🙆 OR 👊 OR 💘 OR 💓 OR 😚 OR ❤ OR 💋 OR 🌹 OR 💕 OR 😙"
emotional_emojis_dict = dict()
emotional_emojis_dict['anger'] = anger
emotional_emojis_dict['anticipation'] = anticipation
emotional_emojis_dict['disgust'] = disgust
emotional_emojis_dict['fear'] = fear
emotional_emojis_dict['joy'] = joy
emotional_emojis_dict['sadeness'] = sadeness
emotional_emojis_dict['surprise'] = surprise
emotional_emojis_dict['trust'] = trust

# twint
nest_asyncio.apply()

tw = twint.Config()
tw.Pandas = True
tw.Store_pandas = True
tw.Pandas_clean = True
tw.Hide_output = True


plus_grandes_villes_de_france = ['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Rennes', 'Reims', 'Saint-Étienne', 'Le Havre', 'Toulon', 'Grenoble', 'Dijon', 'Angers', 'Nîmes', 'Villeurbanne']
# 'Monaco', 'Bruxelles', 'Québec',  'London', 'Dublin', 'Madrid', 'Lisbone', 'Berlin', 'Rome', 'Vienne', 'Luxembourg', 
len(plus_grandes_villes_de_france)


#-------------------------------------------------

# emotional_tweets_dict = dict()

# for each time delta
def fetch_tweets(start_date, end_date, add_time_delta, plus_grandes_villes_de_france, outputDir):
    next_start_date = start_date
    
    tmp_start_date = start_date
    tmp_next_start_date = next_start_date
    total_delta_counter = 0
    while tmp_start_date < end_date:
        tmp_next_start_date = add_time_delta(tmp_next_start_date)
        tmp_start_date = tmp_next_start_date
        total_delta_counter += 1
    print ("INFO: time delta factor = "+str(total_delta_counter))

    
    print ("INFO: start_date: "+str(start_date))
    
    scraping_start = time.time()
    print ("INFO: scraping Twitter starting at: "+str(scraping_start)+"...")
    
    delta_counter = 0
    while start_date < end_date:
        next_start_date = add_time_delta(next_start_date)
    
        day_start = time.time()

        # for each top city
        city_counter = 0
        # SET 
        #for v in plus_grandes_villes_de_france:
        for v in plus_grandes_villes_de_france:
            city_counter += 1
            tw.Near = v
            city_start = time.time()

            # for each affect
            emotion_counter = 0
            for e in emotional_emojis_dict:
                #print ('Debug: e\t',e)
                emotion_counter += 1
                affect_start = time.time()

                tw.Search =  emotional_emojis_dict[e]

                #
                tw.Since = str(start_date)
                tw.Until = str(next_start_date)
            
                #
                twint.run.Search(tw)
                #if e not in emotional_tweets_dict:
                #    emotional_tweets_dict[e] = twint.storage.panda.Tweets_df
                #else:
                #    emotional_tweets_dict[e] = emotional_tweets_dict[e].append(twint.storage.panda.Tweets_df, ignore_index=True)
                current_d_v_e_df = twint.storage.panda.Tweets_df
                affect_end = time.time()
                outputfile = outputDir+"/tweets_"+e+'_'+v+'_'+str(start_date).replace(' ','_')+'_'+str(next_start_date).replace(' ','_')+'.csv' #'+.replace(' ','_')
                print ("INFO: "+str(delta_counter)+"/"+str(total_delta_counter)+"\t"+str(start_date).replace(' ','_')+"\t"+v+"("+str(city_counter)+"/"+str(len(plus_grandes_villes_de_france))+")\t"+e+"("+str(emotion_counter)+"/"+str(len(emotional_emojis_dict))+")\t"+str(affect_end - affect_start))
                if not (current_d_v_e_df.empty):
                    current_d_v_e_df['emotion'] = [e]*len(current_d_v_e_df)
                    current_d_v_e_df['city'] = [v]*len(current_d_v_e_df)
                    current_d_v_e_df['since'] = [tw.Since.split(' ')[0]]*len(current_d_v_e_df)
                
                #print("INFO: scraping affect "+e+" ellapsed time: "+str(affect_end - affect_start))
                    print ("INFO: Printing "+outputfile) #+" ; ellapsed time: "+str(affect_end - affect_start))
                #emotional_tweets_dict[e]
                    current_d_v_e_df.to_csv(outputfile, sep = '\t', index=False) 
            #else: 

            city_end = time.time()
        #print("INFO: scraping city "+v+" ellapsed time: "+str(city_end - city_start))
    
        day_end = time.time()
    #print("INFO: scraping dat "+str(start_date)+" ellapsed time: "+str(day_end - day_start))
    
        start_date = next_start_date
        delta_counter += 1
        print ()
            
    scraping_end = time.time()
    print("INFO: scraping ellapsed time: "+str(scraping_end - scraping_start))
       #tweets_df = pd.DataFrame()
  

In [ ]:
# FIXME how the data_path was populated...

#-------------------------------------------------
# Setting 0
start_date = datetime.datetime(2018,1,1,0,0,0)
end_date = datetime.datetime(2018,1,8,0,0,0)

plus_grandes_villes_de_france = ['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Rennes', 'Reims', 'Saint-Étienne', 'Le Havre', 'Toulon', 'Grenoble', 'Dijon', 'Angers', 'Nîmes', 'Villeurbanne']

def add_time_delta(a_date):
    a_date += datetime.timedelta(days=1)
    return a_date

# 
outputDir = "built/100tweets_top20-city-fr_8emo_20180101-0107"
outputDir = "/tmp/100tweets_top20-city-fr_8emo_20180101-0107"

if not(os.path.isfile(outputDir+".zip")) and not(os.path.isdir(outputDir)):
    os.mkdir(outputDir)

tw.Limit = 100 # 5000 ~ 60 # 1000 ~10s (sans Géo)
tw.Lang = "fr"

# UNcomment to run
fetch_tweets(start_date, end_date, add_time_delta, plus_grandes_villes_de_france, outputDir)


### Comparaison de l'option native de stockage de twing et celle ultérieure via dataframe

Après observations des deux fichiers produits. Avec dataframe, on peut choisir le séparateur et rajouter des colonnes. Il n'y a pas de perte d'information dans aucun des deux cas. Le champ tweet est le même.

In [3]:
import twint
import nest_asyncio
import pandas as pd
import datetime


# twint
nest_asyncio.apply()

tw = twint.Config()
tw.Pandas = True
tw.Store_pandas = True
tw.Pandas_clean = True
tw.Hide_output = True

c = "Paris"
e = "Café"
tw.Limit = 1000 # 1000 ~10s (sans Géo)
tw.Since = str(datetime.datetime(2018,1,1,0,0,0))
tw.Until = str(datetime.datetime(2018,1,10,0,0,0))
tw.Near = c
tw.Search = e
tw.Store_pandas = True
outputName = "/tmp/tweets_"+e+'_'+c+'_'+tw.Since.replace(' ','_')+'_'+tw.Until .replace(' ','_') #+'.csv' #'+
tw.Output = outputName+"_tw.csv"        
twint.run.Search(tw)
current_d_v_e_df = twint.storage.panda.Tweets_df
current_d_v_e_df['emotion'] = [e]*len(current_d_v_e_df)
current_d_v_e_df['city'] = [c]*len(current_d_v_e_df)
current_d_v_e_df['since'] = [tw.Since.split(' ')[0]]*len(current_d_v_e_df)
print (len(current_d_v_e_df))
# current_d_v_e_df.head()
# current_d_v_e_df['tweet'][0]
current_d_v_e_df.to_csv(outputName+"_pd.csv", sep = '\t', index=False) 

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
84


In [ ]:
# sudo pip3 install langdetect
from langdetect import detect

for t in tweet:
    print (t['tweet'])
    #if detect(t['tweet']) == 'fr': 
    #    print(t)